In [102]:
import numpy as np
import time # %%timeit has stats, but doesn't track GC, so use explicit process_time, I'm unsure if %%time tracks GC
np.random.seed(0)

In [103]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [104]:
# Simple example.
X = np.asarray([[0,1,1,0]])
Z = np.asarray([3,2])

# Expected output.
Y_expected = np.asarray([[3,2,2,3]])

In [105]:
def python_loop(X,H,W,Z):
    Y = np.empty_like(X) # Switched to empty_like since we'll use all indices, this saves on zeroing the Y matrix.
    for h in range(H):
        for w in range(W):
            val = X[h,w]
            Y[h,w] = Z[val]
    return Y

In [106]:
Y = python_loop(X,H=X.shape[0],W=X.shape[1],Z=Z)

In [107]:
Y

array([[3, 2, 2, 3]])

In [108]:
assert np.alltrue(Y_expected == Y)

In [109]:
# Bigger example.
H = 5000
W = 5000
X = np.random.randint(6, size=(H,W))
Z = np.asarray([10,11,12,13,14,15])

In [110]:
X

array([[4, 5, 0, ..., 4, 3, 0],
       [2, 2, 1, ..., 1, 0, 5],
       [2, 4, 0, ..., 1, 2, 5],
       ..., 
       [3, 1, 5, ..., 3, 1, 3],
       [2, 3, 3, ..., 2, 5, 5],
       [1, 2, 4, ..., 3, 1, 2]])

In [111]:
Z

array([10, 11, 12, 13, 14, 15])

In [112]:
%%time
Y_loop = python_loop(X,H,W,Z)

CPU times: user 7.74 s, sys: 12 ms, total: 7.75 s
Wall time: 7.75 s


In [113]:
for i in range(10):
    a = time.process_time()
    Y_loop = python_loop(X,H,W,Z)
    b = time.process_time()
    print("Execution time is {} s".format(b-a))

Execution time is 8.040576578000014 s
Execution time is 8.160830093999948 s
Execution time is 7.749624482000058 s
Execution time is 7.843026991999977 s
Execution time is 7.922883432000049 s
Execution time is 7.738561699999991 s
Execution time is 7.864999529999977 s
Execution time is 7.809825566999962 s
Execution time is 7.796143354000037 s
Execution time is 7.699588722000044 s


In [114]:
def vectorize_loop(X,Z):
    """A better? vectorized loop"""
    Y = np.empty_like(X)
    for x, z in enumerate(Z):
        Y[X==x] = z
    return Y

In [115]:
%%time
Y_vec = vectorize_loop(X,Z)

CPU times: user 624 ms, sys: 28 ms, total: 652 ms
Wall time: 649 ms


In [116]:
for i in range(10):
    a = time.process_time()
    Y_vec = vectorize_loop(X,Z)
    b = time.process_time()
    print("Execution time is {} s".format(b-a))

Execution time is 0.6491648260000602 s
Execution time is 0.6235060870000098 s
Execution time is 0.5965811920000306 s
Execution time is 0.6218038189999788 s
Execution time is 0.5884592200000043 s
Execution time is 0.5872754279999981 s
Execution time is 0.6016162140000461 s
Execution time is 0.5924948959999483 s
Execution time is 0.5946380770000133 s
Execution time is 0.5869493039999725 s


In [117]:
assert np.alltrue(Y_vec == Y_loop)

In [118]:
%%cython
def cython_slow_loop(Y,X,H,W,Z):
    """Cython without any datatypes declared. Note the vars are pass-by-ref."""
    for h in range(H):
        for w in range(W):
            Y[h,w] = Z[X[h,w]]
    return Y

In [119]:
%%time
Yc_slow = np.empty_like(X)
cython_slow_loop(Yc_slow,X,H,W,Z)

CPU times: user 6.78 s, sys: 32 ms, total: 6.81 s
Wall time: 6.79 s


In [120]:
for i in range(10):
    a = time.process_time()
    Yc_slow = np.empty_like(X)
    cython_slow_loop(Yc_slow,X,H,W,Z)
    b = time.process_time()
    print("Execution time is {} s".format(b-a))

Execution time is 6.796748275000027 s
Execution time is 6.706175556000062 s
Execution time is 6.7109395769999765 s
Execution time is 6.712201535999952 s
Execution time is 6.666113223999901 s
Execution time is 6.698336114000085 s
Execution time is 6.671143561000008 s
Execution time is 6.656526871999972 s
Execution time is 6.612933903999988 s
Execution time is 6.654387261000011 s


In [121]:
# Make sure values are equal.
assert np.alltrue(Yc_slow == Y_loop)

In [122]:
%%cython
def cython_loop(long [:,:] Y, long [:,:] X, long H, long W, long [:] Z):
    """Real fast cython loop with variable types declared."""
    cdef long h
    cdef long w
    cdef long val
    for h in range(H):
        for w in range(W):
            val = X[h,w]
            Y[h,w] = Z[val]
    return Y

In [123]:
%%time
Yc = np.empty_like(X)
cython_loop(Yc,X,H,W,Z)

CPU times: user 88 ms, sys: 24 ms, total: 112 ms
Wall time: 112 ms


In [124]:
for i in range(10):
    a = time.process_time()
    Yc = np.empty_like(X)
    cython_loop(Yc,X,H,W,Z)
    b = time.process_time()
    print("Execution time is {} s".format(b-a))

Execution time is 0.09039332199995442 s
Execution time is 0.06961140600003546 s
Execution time is 0.07071664600005079 s
Execution time is 0.058919328999991194 s
Execution time is 0.058738907000019935 s
Execution time is 0.058764312999983304 s
Execution time is 0.06657909300008669 s
Execution time is 0.05968121199998677 s
Execution time is 0.059940409999967414 s
Execution time is 0.058434362000070905 s


In [125]:
assert np.alltrue(Yc == Y_loop)

In [126]:
Yc = np.empty_like(X)
for i in range(10):
    a = time.process_time()
    cython_loop(Yc,X,H,W,Z)
    b = time.process_time()
    print("Execution time is {} s".format(b-a))

Execution time is 0.10892011500004628 s
Execution time is 0.067715163999992 s
Execution time is 0.05603121100000408 s
Execution time is 0.05732491799994932 s
Execution time is 0.04924554799993075 s
Execution time is 0.049323712999921554 s
Execution time is 0.05073933100004524 s
Execution time is 0.049617672000067614 s
Execution time is 0.056857606999983545 s
Execution time is 0.06376729600003728 s
